In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [6]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.head(2)

,user_id,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [63]:
items_weights = data_train.groupby('item_id')\
['sales_value'].sum().reset_index().rename(
    columns={'sales_value': 'weight'})
items_weights['weight'] = items_weights['weight'].apply(lambda x: np.log(x))
items_weights['weight'] = softmax(items_weights['weight'])
items_weights.head(2)

,item_id,weight
0,25671,2.969296e-06
1,26081,1.403822e-07


In [64]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    recs = np.random.choice(items_weights['item_id'], n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

#### Использовал функцию softmax, чтобы преобразовать веса в плотность распределения

In [65]:
%%time

result['recommendations'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))

CPU times: user 2.08 s, sys: 26.3 ms, total: 2.1 s
Wall time: 2.14 s


In [66]:
result.head(3)

,user_id,item_id,recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7154535, 13039165, 1098066, 961403, 6034964]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12172355, 9483663, 6534178, 12171206, 844759]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[847049, 12262830, 859075, 918676, 937210]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [67]:
result = pd.read_csv('output/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[9485371, 17104661, 9837998, 12171628, 13777104]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[13910038, 48728, 13416521, 12263605, 828300]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [90]:
def precision_at_k(recommended, bought, n=5):
    recommended = np.array(recommended.strip('[]').split())[:n]
    bought = np.array(bought.strip('[]').split())
    
    flags = np.isin(recommended, bought)
    
    return flags.sum() / len(flags)

In [91]:
metrics = pd.DataFrame(columns=['method', 'precision@5'])

for i in result.iloc[:, 2:]:
    metric = np.array([precision_at_k(pred, bought) for pred, bought in zip(result[i], result['actual'])]).mean()
    metrics.loc[metrics.shape[0]+1] = [i, metric]

In [92]:
metrics

,method,precision@5
1,random_recommendation,0.000098
2,popular_recommendation,0.069638
3,itemitem,0.019197
4,cosine,0.019001
5,tfidf,0.018511
6,own_purchases,0.032419
